In [2]:
# general imports
import glob
import os
from datetime import datetime
from dateutil.parser import parse
import shutil

# download
import drms
from urllib import request

# data processing
import numpy as np
from astropy.nddata import block_reduce
from sunpy.map import Map

# deep learning
import torch

# NF2
from nf2.train.trainer import NF2Trainer
from nf2.data.download import download_HARP, find_HARP, donwload_ds
from nf2.train.metric import *
from nf2.evaluation.unpack import load_cube
from nf2.evaluation.energy import get_free_mag_energy
from nf2.data.loader import load_hmi_data

# visualization
from matplotlib import pyplot as plt

In [2]:
import torch

In [3]:
print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
!python -c 'import platform;print(platform.platform())'

PyTorch version:1.13.1
MPS 장치를 지원하도록 build 되었는지: True
MPS 장치가 사용 가능한지: True
macOS-12.6.3-arm64-arm-64bit


In [4]:
device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

In [5]:
nf2_path = './database/author/example_7115.nf2'

In [7]:
state = torch.load(nf2_path, map_location=device)

In [8]:
state

{'model': BModel(
   (d_in): Linear(in_features=3, out_features=256, bias=True)
   (linear_layers): ModuleList(
     (0): Linear(in_features=256, out_features=256, bias=True)
     (1): Linear(in_features=256, out_features=256, bias=True)
     (2): Linear(in_features=256, out_features=256, bias=True)
     (3): Linear(in_features=256, out_features=256, bias=True)
     (4): Linear(in_features=256, out_features=256, bias=True)
     (5): Linear(in_features=256, out_features=256, bias=True)
     (6): Linear(in_features=256, out_features=256, bias=True)
     (7): Linear(in_features=256, out_features=256, bias=True)
   )
   (d_out): Linear(in_features=256, out_features=3, bias=True)
   (activation): Sine()
 ),
 'cube_shape': [344, 224, 160],
 'b_norm': 2500,
 'spatial_norm': 160,
 'meta_info': MetaDict([('simple', True),
           ('bitpix', 32),
           ('naxis', 2),
           ('naxis1', 688),
           ('naxis2', 448),
           ('pcount', 0),
           ('gcount', 1),
           ('xt